In [34]:
import pandas as pd
import numpy as np
from glob import glob
import ahocorasick
import os
import sys
import seaborn as sn
import matplotlib.pyplot as plt
import re

In [51]:
def get_info(files):
    peps_tissues = {}
    for f in files:
        tissue_peps = [x.strip() for x in open(f, 'r').readlines()]
    
        for tissue_pep in tissue_peps[1::]:
            tissue = tissue_pep.split('\t')[0].lower()
            peps  = tissue_pep.split('\t')[1].upper()
            for pep in peps.split(','):
                try:
                    peps_tissues[pep].add(tissue)
                except KeyError:
                    peps_tissues[pep] = set([tissue])
    return peps_tissues

peps_tissues = get_info(['count-tables/PXD016999-tissue_peps.tsv', 'count-tables/PXD010154-tissue_peps.tsv'])
peps_samples = get_info(['count-tables/sample_peps.tsv'])
print(len(peps_samples))
print(len(peps_tissues))



4991
4991


In [56]:
peps_file = 'blast_canonical-count-tables/1-2mismatches_peptides_match_info/peptide_gene_symbol_protein_matching_info_add.tsv'
peps_header = [x.strip().split('\t') for x in open(peps_file, 'r').readlines()][0]
peps_all = [x.strip().split('\t') for x in open(peps_file, 'r').readlines()][1::]

pep_var_info = {}
for i, pep_line in enumerate(peps_all):
    pep_seq = pep_line[peps_header.index('peptide')]
    prot_id = pep_line[peps_header.index('canonical_protein_ID')]
    variant_info = pep_line[peps_header.index('mismatch_info')]
    gene_symbol = pep_line[peps_header.index('gene_symbol')]
    
    var = re.sub('[0-9]+', '', variant_info)
    var_pos = re.sub('[A-Z]', '', variant_info).strip('>')
    #print('found {} in {} {} {}'.format(pep_seq, prot_id, var, var_pos))
    
    samples = pep_line[peps_header.index('GCA_protein_seq')].split('[')[3].split(']')[0].replace(" ", "").replace("'", "").split(",")
    tissues = set()
    try:
        tissues = set(peps_tissues[pep_seq])
    except KeyError:
        pass
    
    try:
        pep_var_info[pep_seq][0].append(var)
        pep_var_info[pep_seq][1].append(variant_info)
        pep_var_info[pep_seq][2].union(samples)
        pep_var_info[pep_seq][3].union(tissues)
        pep_var_info[pep_seq][3].union(prot_id.split(','))
        pep_var_info[pep_seq][3].union(set(gene_symbol.split(',')))
    except KeyError:
        pep_var_info[pep_seq] = [[var], [variant_info], set(samples), set(tissues), set(prot_id.split(',')), set(gene_symbol.split(','))]
print(len(pep_var_info))


4766


In [57]:
pep_var_info_sorted = {}
for k in sorted(pep_var_info, key=len, reverse=True):
    pep_var_info_sorted[k] = pep_var_info[k]

inserted_pairs = {}
matching_peps = {}
for pep_seq in pep_var_info_sorted.keys():
    p1 = pep_seq
    for s in matching_peps.keys():
        if pep_seq in s:# and pep_var_info_sorted[pep_seq][0]==pep_var_info_sorted[s][0]:
            p1 = s
            break
    
    if p1==pep_seq:
        for pep in pep_var_info_sorted.keys():
            if ((pep_seq in pep) and pep_var_info_sorted[pep_seq][0]==pep_var_info_sorted[pep][0]):
                p1 = pep
                break
    
    try:
        matching_peps[p1].append(pep_seq)
    except KeyError:
        matching_peps[p1] = [pep_seq]

print(len(matching_peps))
#print(matching_peps)



3780


In [54]:
matching_peps_merged = {}
for pep_key, peps in matching_peps.items(): 
    var  = pep_var_info_sorted[pep_key][0]
    gene = pep_var_info_sorted[pep_key][-1]
    
    for p in matching_peps.keys():
        if p == pep_key or pep_key in matching_peps_merged.values():
            continue
        if ( pep_var_info_sorted[p][0] == pep_var_info_sorted[pep_key][0] and
            (set(pep_var_info_sorted[p][1]).issubset(set(pep_var_info_sorted[pep_key][1])) or 
            set(pep_var_info_sorted[pep_key][1]).issubset(set(pep_var_info_sorted[p][1]))) and
            pep_var_info_sorted[p][-1] == pep_var_info_sorted[pep_key][-1] ):
            if len(set(pep_key).intersection(set(p)))>3:
                try:
                    matching_peps_merged[pep_key].extend(matching_peps[p])
                except KeyError:
                    matching_peps_merged[pep_key]= matching_peps[pep_key][0::]
                    matching_peps_merged[pep_key].extend(matching_peps[p])

for pep_key in matching_peps.keys():
    keys = matching_peps_merged.keys()
    items = matching_peps_merged.values()
    
    if pep_key not in items and pep_key not in keys:
        matching_peps_merged[pep_key] = matching_peps[pep_key]
    


In [60]:
samples_var_dict = {} 
genes_var_dict = {}
tissues_var_dict = {}
items_added = []

with open('blast_canonical-count-tables/1-2mismatches_peptides_match_info/peps_merged_vars.tsv', 'w') as out:
    for pep_key, peps in matching_peps_merged.items():
        peps_u = set(sorted(peps))
        if peps_u in items_added:
            continue
        items_added.append(peps_u)
        
        var = []
        for v in pep_var_info_sorted[pep_key][0]:
            var.extend(v.split(','))
        out.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(pep_key, len(peps_u), len(set(samples)), len(set(tissues)), ','.join(set(var)), ','.join(set(gene)), ','.join(peps_u), ','.join(set(tissues)), ))
        
        genes = []
        tissues = []
        samples = []
        for pep in peps_u:
            samples.extend(pep_var_info_sorted[pep][2])
            tissues.extend(pep_var_info_sorted[pep][3])
            genes.extend(pep_var_info_sorted[pep][-1])
        
        for s in set(samples):
            try:
                samples_var_dict[s][0].extend(var)
                samples_var_dict[s][1].extend(list(peps_u))
            except KeyError:
                samples_var_dict[s] = [var, list(peps_u)]

        for s in set(tissues):
            try:
                tissues_var_dict[s][0].extend(var)
                tissues_var_dict[s][1].extend(list(peps_u))
            except KeyError:
                tissues_var_dict[s] = [var, list(peps_u)]
        
        for s in set(genes):
            try:
                genes_var_dict[s][0].extend(var)
                genes_var_dict[s][1].extend(list(peps_u))
            except KeyError:
                genes_var_dict[s] = [var, list(peps_u)]



#make a sample file, number of variants per sample
with open('blast_canonical-count-tables/1-2mismatches_peptides_match_info/samples_merged_pep_vars.tsv', 'w') as outs:
    for s, i in samples_var_dict.items():
        outs.write('{}\t{}\t{}\t{}\t{}\n'.format(s, len(set(i[0])), len(set(i[1])), ','.join(set(i[0])), ','.join(set(i[1])) ))

#make a gene file, number of variants per gene, supprted by #PSMs and peptides
with open('blast_canonical-count-tables/1-2mismatches_peptides_match_info/genes_merged_pep_vars.tsv', 'w') as outg:
    for s, i in genes_var_dict.items():
        outg.write('{}\t{}\t{}\t{}\t{}\n'.format(s, len(set(i[0])), len(set(i[1])), ','.join(set(i[0])), ','.join(set(i[1])) ))

#make a tissue file, number of variatns and number of gene variants per tissue
with open('blast_canonical-count-tables/1-2mismatches_peptides_match_info/tissues_merged_pep_vars.tsv', 'w') as outg:
    for s, i in tissues_var_dict.items():
        outg.write('{}\t{}\t{}\t{}\t{}\n'.format(s, len(set(i[0])), len(set(i[1])), ','.join(set(i[0])), ','.join(set(i[1])) ))
